# Analytics

#### Date: 2020/02

#### SUMMARY:

- This notebook represents the project quality analysis of the date exposed right above. 

### TEAM:

##### Semester: 2020/02
##### Professor: Hilmer Neri

##### Members:

- Member x
- Member y

### LIBRARIES

In [1]:
# Deal with data
import pandas as pd
import numpy as np
import json

# Deal with API request
import urllib3
from urllib3 import request

# Deal with visualization
import seaborn as sns
import matplotlib.pyplot as plt

### GRAPH SETTINGS

In [2]:
%config InlineBackend.figure_format ='retina'
sns.set(font_scale=1.5)
sns.set_style('darkgrid',
              {'xtick.bottom' : True,
               'ytick.left': True,
               'grid.linestyle':'--',
               'font.monospace': ['Computer Modern Typewriter'],
               'axes.edgecolor' : 'white'})

### DATAFRAME SETTINGS

In [3]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

### SonarCloud

#### KEYS

In [4]:
front_key = 'fga-eps-mds_2020.1-VC_Gestor-FrontEnd'

In [5]:
back_key = 'fga-eps-mds_2020.1-VC_Gestor-BackEnd'

#### METRICS

In [6]:
metric_list = ['files',
               'functions',
               'complexity',
               'ncloc',
               'comment_lines',
               'comment_lines_density',
               'duplicated_lines_density',
               'test_success_density',
               'coverage',
               'bugs',
               'reliability_rating',
               'vulnerabilities',
               'security_rating',
               'open_issues',
               'sqale_rating']

len(metric_list)

15

In [7]:
def generate_metric_string(metric_list):
    metric_str = ''
    
    for metric in metric_list:
        metric_str += metric + ','
        
    return metric_str

In [8]:
metric_str = generate_metric_string(metric_list)

#### URLS

In [9]:
front_url = f"https://sonarcloud.io/api/measures/component_tree?component={front_key}&metricKeys={metric_str}&ps=500"
back_url = f"https://sonarcloud.io/api/measures/component_tree?component={back_key}&metricKeys={metric_str}&ps=500"

#### API REQUEST

In [10]:
http = urllib3.PoolManager()

In [11]:
front_request = http.request('GET', front_url)
front_request.status

200

In [12]:
back_request = http.request('GET', back_url)
back_request.status

200

#### JSON DECODING

In [13]:
front_json = json.loads(front_request.data.decode('utf-8'))
back_json = json.loads(back_request.data.decode('utf-8'))

## DATA

### PROJECT

In [14]:
project_front_json = front_json['baseComponent']['measures']
project_back_json = back_json['baseComponent']['measures']

In [15]:
project_front_data = pd.DataFrame(project_front_json)
project_back_data = pd.DataFrame(project_back_json)

##### FRONT

In [16]:
project_front_data

,metric,value,bestValue
0,duplicated_lines_density,4.3,False
1,functions,263,NaN
2,sqale_rating,1.0,True
3,security_rating,1.0,True
4,comment_lines,172,NaN
5,files,45,NaN
6,complexity,412,NaN
7,bugs,8,False
8,ncloc,3509,NaN
9,coverage,41.1,False


##### BACK

In [17]:
project_back_data

,metric,value,bestValue
0,duplicated_lines_density,48.0,False
1,functions,91,NaN
2,sqale_rating,1.0,True
3,security_rating,1.0,True
4,comment_lines,135,NaN
5,files,80,NaN
6,complexity,173,NaN
7,bugs,207,False
8,ncloc,12936,NaN
9,coverage,91.7,False


### FILES

In [18]:
def metric_per_file(json):
    file_json = []
    
    for component in json['components']:
        if component['qualifier'] == 'FIL':
            file_json.append(component)
            
    return file_json

In [19]:
front_file_json = metric_per_file(front_json)
back_file_json = metric_per_file(back_json)

In [20]:
def generate_file_dataframe(metric_list, json, language_extension):
    df_columns = metric_list
    df = pd.DataFrame(columns = df_columns)
    
    for file in json:
        try:
            if file['language'] == language_extension:
                for measure in file['measures']:
                    df.at[file['path'], measure['metric']] = measure['value']
        except:
            pass
        
    df.reset_index(inplace = True)
    df.drop(['files'], axis=1)
    df = df.rename({'index': 'path'}, axis=1)

    return df

In [21]:
front_files_data = generate_file_dataframe(metric_list, front_file_json, language_extension = 'js')
back_files_data = generate_file_dataframe(metric_list, back_file_json, language_extension = 'js')

##### FRONT

In [22]:
front_files_data

,path,files,functions,complexity,ncloc,comment_lines,comment_lines_density,duplicated_lines_density,test_success_density,coverage,bugs,reliability_rating,vulnerabilities,security_rating,open_issues,sqale_rating
0,src/components/404/404.js,1,1,1,14,0,0.0,0.0,100.0,0.0,0,1.0,0,1.0,2,1.0
1,src/components/pages/Anuncio.js,1,5,7,120,13,9.8,10.9,100.0,0.0,0,1.0,0,1.0,13,1.0
2,src/services/apiBeneficio.js,1,0,0,5,0,0.0,0.0,100.0,0.0,0,1.0,0,1.0,0,1.0
3,src/services/apiNoticias.js,1,0,0,5,0,0.0,0.0,100.0,100.0,0,1.0,0,1.0,0,1.0
4,src/services/apiPostagem.js,1,0,0,5,0,0.0,0.0,100.0,100.0,0,1.0,0,1.0,0,1.0
5,src/services/apiUser.js,1,0,0,5,0,0.0,0.0,100.0,0.0,0,1.0,0,1.0,0,1.0
6,src/components/app/App.test.js,1,1,1,8,0,0.0,0.0,100.0,0.0,0,1.0,0,1.0,0,1.0
7,src/components/pages/BeneficiosCriar.js,1,1,1,20,0,0.0,0.0,100.0,0.0,0,1.0,0,1.0,2,1.0
8,src/components/pages/BeneficiosEditar.js,1,1,1,20,0,0.0,0.0,100.0,0.0,0,1.0,0,1.0,0,1.0
9,src/components/pages/BeneficiosGerenciar.js,1,8,10,53,3,5.4,0.0,100.0,0.0,0,1.0,0,1.0,8,1.0


##### BACK

In [23]:
back_files_data

,path,files,functions,complexity,ncloc,comment_lines,comment_lines_density,duplicated_lines_density,test_success_density,coverage,bugs,reliability_rating,vulnerabilities,security_rating,open_issues,sqale_rating
0,packages/user/src/controller/AuthenticateUserS...,1,1,1,11,1,8.3,0.0,100.0,0.0,0,1.0,0,1.0,0,1.0
1,packages/user/src/controller/AuthResolve.js,1,1,3,22,1,4.3,0.0,100.0,100.0,0,1.0,0,1.0,0,1.0
2,packages/benefits/src/models/benefit.js,1,0,0,30,0,0.0,0.0,100.0,100.0,0,1.0,0,1.0,0,1.0
3,packages/benefits/src/controller/BenefitContro...,1,5,5,44,18,29.0,0.0,100.0,28.0,0,1.0,0,1.0,0,1.0
4,packages/benefits/src/test/BenefitsController....,1,12,12,251,0,0.0,0.0,100.0,100.0,0,1.0,0,1.0,0,1.0
5,packages/user/src/controller/ChangePasswordRes...,1,1,5,29,0,0.0,0.0,100.0,100.0,0,1.0,0,1.0,0,1.0
6,packages/user/src/controller/CheckCodeResolve.js,1,1,4,23,0,0.0,0.0,100.0,100.0,0,1.0,0,1.0,0,1.0
7,packages/news/src/test/ControllerTesting.spec.js,1,13,13,316,0,0.0,0.0,100.0,100.0,0,1.0,0,1.0,0,1.0
8,packages/user/src/test/ControllerTesting.spec.js,1,24,24,575,0,0.0,33.4,100.0,100.0,0,1.0,0,1.0,0,1.0
9,packages/benefits/src/controller/CreateBenefit...,1,1,4,16,0,0.0,0.0,100.0,100.0,0,1.0,0,1.0,0,1.0


### ANALYSIS

#### GRAPHS